In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [2]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=3,
    lora_dropout=0.1,
    bias='lora_only',
    target_modules=['query', 'key', 'value', 'attention.output.dense'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 704,260 || all params: 125,316,104 || trainable%: 0.5620


In [3]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [4]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=3000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.363400,1.319008,0.734375
200,1.038000,0.459037,0.862500
300,0.402700,0.358042,0.884375
400,0.299700,0.349030,0.889062
500,0.319900,0.335329,0.898438
600,0.308800,0.332608,0.900000
700,0.308300,0.335975,0.898438
800,0.338200,0.319524,0.901563
900,0.306700,0.315760,0.904687
1000,0.297400,0.312396,0.906250


In [7]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2800'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [01:00<00:00,  4.14it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2800.csv


In [5]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-3000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3634,1.319008,0.380625,0.734375,0.000048,0.013405,0.044392,1.033655,-0.353750,0.518298
1,200,1.0380,0.459037,0.744643,0.862500,0.000047,0.026810,0.578963,2.261255,-0.117857,0.863354
2,300,0.4027,0.358042,0.868750,0.884375,0.000045,0.040214,0.044658,1.124727,-0.015625,0.982332
3,400,0.2997,0.349030,0.898661,0.889062,0.000043,0.053619,-0.049330,0.858665,0.009598,1.010796
4,500,0.3199,0.335329,0.894643,0.898438,0.000042,0.067024,-0.015429,0.953989,-0.003795,0.995776
5,600,0.3088,0.332608,0.894196,0.900000,0.000040,0.080429,-0.023808,0.928421,-0.005804,0.993552
6,700,0.3083,0.335975,0.898661,0.898438,0.000038,0.093834,-0.027675,0.917626,0.000223,1.000248
7,800,0.3382,0.319524,0.887946,0.901563,0.000037,0.107239,0.018676,1.058449,-0.013616,0.984897
8,900,0.3067,0.315760,0.897768,0.904687,0.000035,0.120643,-0.009060,0.971308,-0.006920,0.992351
9,1000,0.2974,0.312396,0.900893,0.906250,0.000033,0.134048,-0.014996,0.951997,-0.005357,0.994089
